# Modelos de reapertura modular

### Paquetes

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import EoN
from matplotlib import rc
import scipy
import random
import pandas as pd
import numpy as np
from collections import defaultdict

### Definir parámetros epidemiológicos

#### Función para preparar la red para modelo epidemiológico

### Modelo Con Parámetros de JNSD

### Modelo con Reapertura Total

### Evaluar Reapertura aleatoria, diferentes niveles de población reintegrándose

### Evaluar reapertura modular 